In [20]:
import pandas as pd
import json,natsort, os
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [21]:
plt.rc('xtick', labelsize=120)    # fontsize of the tick labels
plt.rc('ytick', labelsize=120)    # fontsize of the tick labels
plt.rc('legend', fontsize=120)    # legend fontsize
# increase size of axis label size
plt.rcParams['axes.labelsize'] = 150
# increase space between axis label and axis
plt.rcParams['axes.labelpad'] = 60
# increase title size
plt.rcParams['axes.titlesize'] = 150
# increase space between title and plot
plt.rcParams['axes.titlepad'] = 60
# increase padding between ticks and their labels
plt.rcParams['xtick.major.pad'] = 60
plt.rcParams['ytick.major.pad'] = 60

In [22]:
hw_data_file = '/mydata/iiswc24/lotus/code/image_classification/analysis/combine_lotus/combined_lotustrace_uarch.csv'
high_level_data_file = '/mydata/iiswc24/lotus/code/image_classification/analysis/combine_lotus/all_config_stacked.csv'
mapping_file = '/mydata/iiswc24/lotus/code/image_classification/LotusMap/Intel/mapping_funcs.json'
plot_dir = '/mydata/iiswc24/lotus/code/image_classification/analysis/combine_lotus/mapped_python_figs'

os.makedirs(plot_dir, exist_ok=True)

In [23]:
# read data as csv
hw_df = pd.read_csv(hw_data_file, index_col=0)
high_level_df = pd.read_csv(high_level_data_file)

In [24]:
func_rename = {
    'c10::function_ref<void (char**, long const*, long, long)>::callback_fn<at::native::AVX2::VectorizedLoop2d<at::native::AVX2::direct_copy_kernel(at::TensorIteratorBase&)::{lambda()#3}::operator()(void) const::{lambda()#7}::operator()(void) const::{lambda(float)#1}, at::native::AVX2::direct_copy_kernel(at::TensorIteratorBase&)::{lambda()#3}::operator()(void) const::{lambda()#7}::operator()(void) const::{lambda(at::vec::AVX2::Vectorized<float>)#2}>>': 'AVX2::direct_copy_kernel(float)',
    'at::native::AVX2::vectorized_loop<at::native::(anonymous namespace)::div_true_kernel(at::TensorIteratorBase&)::{lambda()#1}::operator()(void) const::{lambda()#2}::operator()(void) const::{lambda(floatfloat)#1}&, at::native::(anonymous namespace)::div_true_kernel(at::TensorIteratorBase&)::{lambda()#1}::operator()(void) const::{lambda()#2}::operator()(void) const::{lambda(at::vec::AVX2::Vectorized<float>at::vec::AVX2::Vectorized<float>)#2}&>.isra.0' :'AVX2::div_true_kernel(float)',
    'c10::function_ref<void (char**, long const*, long, long)>::callback_fn<at::native::AVX2::VectorizedLoop2d<at::native::AVX2::direct_copy_kernel(at::TensorIteratorBase&)::{lambda()#3}::operator()(void) const::{lambda()#1}::operator()(void) const::{lambda(unsigned char)#1}, at::native::AVX2::direct_copy_kernel(at::TensorIteratorBase&)::{lambda()#3}::operator()(void) const::{lambda()#1}::operator()(void) const::{lambda(at::vec::AVX2::Vectorized<unsigned char>)#2}>>' : 'AVX2::direct_copy_kernel(unsigned char)',
    'c10::function_ref<void (char**, long const*, long, long)>::callback_fn<at::TensorIteratorBase::loop_2d_from_1d<at::native::AVX2::copy_kernel(at::TensorIterator&, bool)::{lambda()#1}::operator()(void) const::{lambda()#7}::operator()(void) const::{lambda()#1}::operator()(void) const::{lambda()#1}::operator()(void) const::{lambda(char**long const*, long)#1}>(, signed char, at::native::AVX2::copy_kernel(at::TensorIterator&, bool)::{lambda()#1}::operator()(void) const::{lambda()#7}::operator()(void) const::{lambda()#1}::operator()(void) const::{lambda()#1}::operator()(void) const::{lambda(char**long const*, long)#1} const&)::{lambda(char**long const*, long, long)#1}>' : 'AVX2::copy_kernel(char**long const*, long, long)',
    'c10::function_ref<void (char**, long const*, long, long)>::callback_fn<at::native::AVX2::VectorizedLoop2d<at::native::(anonymous namespace)::add_kernel(at::TensorIteratorBase&, c10::Scalar const&)::{lambda()#1}::operator()(void) const::{lambda()#7}::operator()(void) const::{lambda(floatfloat)#1}, at::native::(anonymous namespace)::add_kernel(at::TensorIteratorBase&, c10::Scalar const&)::{lambda()#1}::operator()(void) const::{lambda()#7}::operator()(void) const::{lambda(at::vec::AVX2::Vectorized<float>at::vec::AVX2::Vectorized<float>)#2}>>' : 'add_kernel(float)',
}

In [25]:
hw_df.head()

,Function,CPU Time (s),Retiring,Front-End Bound,Bad Speculation,L1 Bound,L2 Bound,L3 Bound,Memory Bandwidth,Local DRAM,Remote DRAM,Remote Cache,Store Bound,Core Bound,CPU Time %,config
0,decode_mcu,2215.113,8.262188e+13,1.388353e+13,2.007549e+13,1.538038e+13,2.397071e+12,3.444859e+12,2.327759e+13,1.286107e+13,4.095811e+12,1.411467e+10,1.155543e+13,8.408220e+12,18.711362,20
1,jpeg_idct_islow,1894.678,9.659520e+13,1.623157e+13,2.347073e+13,1.798157e+13,2.802473e+12,4.027467e+12,2.721438e+13,1.503619e+13,4.788509e+12,1.650180e+10,1.350972e+13,9.830249e+12,16.004604,20
2,ImagingResampleHorizontal_8bpc,900.121,2.033247e+14,3.416607e+13,4.940389e+13,3.784968e+13,5.898967e+12,8.477474e+12,5.728395e+13,3.164989e+13,1.007940e+13,3.473489e+10,2.843681e+13,2.069184e+13,7.603445,20
3,ycc_rgb_convert,751.321,2.435934e+14,4.093270e+13,5.918839e+13,4.534585e+13,7.067264e+12,1.015645e+13,6.862910e+13,3.791819e+13,1.207564e+13,4.161417e+10,3.406876e+13,2.478988e+13,6.346511,20
4,ImagingResampleVertical_8bpc,621.411,2.945181e+14,4.948995e+13,7.156210e+13,5.482570e+13,8.544721e+12,1.227972e+13,8.297646e+13,4.584523e+13,1.460013e+13,5.031388e+10,4.119105e+13,2.997237e+13,5.249144,20


In [26]:
high_level_df.head()

,config,Collation,Loader,Normalize,RandomHorizontalFlip,RandomResizedCrop,ToTensor
0,20,524.543835,6022.697503,277.400632,78.403272,1418.447922,430.454935


In [27]:
# make config column the index
high_level_df = high_level_df.set_index('config')

# map config column with format 'b1024_gpu4_dataloader8' to '8' casted to int
# hw_df['config'] = hw_df['config'].map(lambda x: int(x.split('_')[-1].replace('dataloader', '')))

In [28]:
print('Numbers are %s (not multiplied by 100)')
high_level_df.head()

Numbers are %s (not multiplied by 100)


,Collation,Loader,Normalize,RandomHorizontalFlip,RandomResizedCrop,ToTensor
config,,,,,,
20,524.543835,6022.697503,277.400632,78.403272,1418.447922,430.454935


In [29]:
hw_df.head()

,Function,CPU Time (s),Retiring,Front-End Bound,Bad Speculation,L1 Bound,L2 Bound,L3 Bound,Memory Bandwidth,Local DRAM,Remote DRAM,Remote Cache,Store Bound,Core Bound,CPU Time %,config
0,decode_mcu,2215.113,8.262188e+13,1.388353e+13,2.007549e+13,1.538038e+13,2.397071e+12,3.444859e+12,2.327759e+13,1.286107e+13,4.095811e+12,1.411467e+10,1.155543e+13,8.408220e+12,18.711362,20
1,jpeg_idct_islow,1894.678,9.659520e+13,1.623157e+13,2.347073e+13,1.798157e+13,2.802473e+12,4.027467e+12,2.721438e+13,1.503619e+13,4.788509e+12,1.650180e+10,1.350972e+13,9.830249e+12,16.004604,20
2,ImagingResampleHorizontal_8bpc,900.121,2.033247e+14,3.416607e+13,4.940389e+13,3.784968e+13,5.898967e+12,8.477474e+12,5.728395e+13,3.164989e+13,1.007940e+13,3.473489e+10,2.843681e+13,2.069184e+13,7.603445,20
3,ycc_rgb_convert,751.321,2.435934e+14,4.093270e+13,5.918839e+13,4.534585e+13,7.067264e+12,1.015645e+13,6.862910e+13,3.791819e+13,1.207564e+13,4.161417e+10,3.406876e+13,2.478988e+13,6.346511,20
4,ImagingResampleVertical_8bpc,621.411,2.945181e+14,4.948995e+13,7.156210e+13,5.482570e+13,8.544721e+12,1.227972e+13,8.297646e+13,4.584523e+13,1.460013e+13,5.031388e+10,4.119105e+13,2.997237e+13,5.249144,20


In [30]:
# load the mapping functions
mapping_funcs = json.load(open(mapping_file))


In [31]:
mapping_funcs

{'op_to_func': {'RandomHorizontalFlip': ['__libc_calloc|libc.so.6',
   '__GI___pthread_mutex_lock|libpthread.so.0',
   '_PyEval_EvalFrameDefault|python3.10',
   'ImagingFlipLeftRight|_imaging.cpython-310-x86_64-linux-gnu.so',
   '__GI___libc_malloc|libc.so.6',
   'c10::impl::OperatorEntry::lookup|libtorch_cpu.so'],
  'Loader': ['__libc_calloc|libc.so.6',
   'read|python3.10',
   '__memmove_avx_unaligned_erms|libc.so.6',
   '_int_free|libc.so.6',
   '__GI___libc_malloc|libc.so.6',
   'PyEval_RestoreThread|python3.10',
   'vgetargs1_impl|python3.10',
   '__memset_avx2_unaligned_erms|libc.so.6',
   'ImagingJpegDecode|_imaging.cpython-310-x86_64-linux-gnu.so',
   'PyVectorcall_Function|python3.10',
   'decode_mcu|libjpeg.so.9',
   'PyThread_get_thread_ident|python3.10',
   'ycc_rgb_convert|libjpeg.so.9',
   '_Py_NewReference|python3.10',
   '__libc_read|libpthread.so.0',
   'ImagingUnpackRGB|_imaging.cpython-310-x86_64-linux-gnu.so',
   'jpeg_idct_16x16|libjpeg.so.9',
   'decompress_onepas

In [32]:
# get CPU Time of '_Py_INCREF' function in hw_df
hw_df[hw_df['Function'] == '_Py_INCREF']


,Function,CPU Time (s),Retiring,Front-End Bound,Bad Speculation,L1 Bound,L2 Bound,L3 Bound,Memory Bandwidth,Local DRAM,Remote DRAM,Remote Cache,Store Bound,Core Bound,CPU Time %,config
16,_Py_INCREF,55.38,3.304746e+15,5.553195e+14,8.029880e+14,6.151912e+14,9.587908e+13,1.377890e+14,9.310669e+14,5.144227e+14,1.638260e+14,5.645648e+11,4.621989e+14,3.363156e+14,0.467802,20


In [33]:
# print columns of hw_df
hw_df.columns

Index(['Function', 'CPU Time (s)', 'Retiring', 'Front-End Bound',
       'Bad Speculation', 'L1 Bound', 'L2 Bound', 'L3 Bound',
       'Memory Bandwidth', 'Local DRAM', 'Remote DRAM', 'Remote Cache',
       'Store Bound', 'Core Bound', 'CPU Time %', 'config'],
      dtype='object')

In [34]:
hw_columns_to_su = ['CPU Time (s)', 'Retiring', 'Front-End Bound',
       'Bad Speculation', 'L1 Bound', 'L2 Bound', 'L3 Bound',
       'Memory Bandwidth', 'Local DRAM', 'Remote DRAM', 'Remote Cache',
       'Store Bound', 'Core Bound', 'CPU Time %']
combined_df = pd.DataFrame()
# loop config column in high_level_df
for config in high_level_df.index:
    print('Config: ',config)
    config_df = pd.DataFrame()
    # loop through each column in high_level_df
    for python_func in high_level_df.columns:
        print('\tPython func: ',python_func)
        # get the mapping function for the python_func
        c_cppfuncs = mapping_funcs['op_to_func'][python_func]
        python_df = pd.DataFrame()
        # loop through each column in mapping_func
        for c in c_cppfuncs:
            # rename the c func if it exists in the func_rename dict
            func_only = c.split('|')[0]
            if func_only in func_rename.keys():
                func_only = func_rename[func_only]
            print('\t\tC/C++ func: ',func_only)
            # get the column in hw_df that matches the c func and config
            hw_df_col = hw_df[(hw_df['config'] == config) & (hw_df['Function'] == func_only)].copy()
            # check if empty
            if hw_df_col.empty:
                print('\t\t\tEmpty')
                continue
            # print('\t\t\tHW data: ',hw_df_col)
            # drop the config and function column
            hw_df_col = hw_df_col.drop(columns=['config', 'Function'])
            # multiply the hw_df_col by the percentage of the python_func if its a common op
            if c in mapping_funcs['common_funcs_to_op']:
                ops = mapping_funcs['common_funcs_to_op'][c]
                base_total = 0
                for op in ops:
                    base_total += high_level_df.loc[config, op]
                scale_factor = high_level_df.loc[config, python_func] / base_total
                print('\t\t\tScale factor: ',scale_factor)
                hw_df_col = hw_df_col.mul(scale_factor)
            # concat the hw_df_col to the python_df
            python_df = pd.concat([python_df, hw_df_col])
        # sum the python_df and print
        python_series = python_df.sum()
        python_df = pd.DataFrame({python_func: python_series})
        # concat the python_df to the config_df
        config_df = pd.concat([config_df, python_df],axis=1)

    # transpose the config_df
    config_df = config_df.T
    # make index a column with name 'Function'
    config_df = config_df.reset_index().rename(columns={'index': 'Function'})
    # add config column to config_df
    config_df['config'] = config
    # concat the config_df to the combined_df
    combined_df = pd.concat([combined_df, config_df])
# reset the index
combined_df = combined_df.reset_index(drop=True)

Config:  20
	Python func:  Collation
		C/C++ func:  GOMP_parallel
			Empty
		C/C++ func:  __kmp_join_barrier
			Empty
		C/C++ func:  __GI___pthread_getspecific
			Empty
		C/C++ func:  OS_BARESYSCALL_DoCallAsmIntel64Linux
			Empty
		C/C++ func:  _int_free
			Scale factor:  0.06247437193537265
		C/C++ func:  [Outside any known module]
			Empty
		C/C++ func:  __tls_get_addr
		C/C++ func:  at::native::structured_cat_out_cpu::impl
			Empty
		C/C++ func:  __kmp_fork_barrier
			Empty
		C/C++ func:  [OpenMP dispatcher]
			Empty
		C/C++ func:  pymalloc_alloc
		C/C++ func:  AVX2::direct_copy_kernel(float)
			Scale factor:  0.6540899737119084
		C/C++ func:  __kmp_GOMP_microtask_wrapper
			Empty
		C/C++ func:  c10::TensorImpl::data
			Empty
		C/C++ func:  __kmp_finish_implicit_task
			Empty
	Python func:  Loader
		C/C++ func:  __libc_calloc
			Scale factor:  0.987149323592662
		C/C++ func:  read
			Empty
		C/C++ func:  __memmove_avx_unaligned_erms
			Scale factor:  0.7651172858608224
		C/C++ func:

In [35]:
combined_df.head()

,Function,CPU Time (s),Retiring,Front-End Bound,Bad Speculation,L1 Bound,L2 Bound,L3 Bound,Memory Bandwidth,Local DRAM,Remote DRAM,Remote Cache,Store Bound,Core Bound,CPU Time %,config
0,Collation,206.538240,2.594403e+16,4.359557e+15,6.303888e+15,4.829582e+15,7.527024e+14,1.081717e+15,7.309376e+15,4.038495e+15,1.286122e+15,4.432138e+12,3.628510e+15,2.640258e+15,1.744657,20
1,Loader,6454.017994,3.761590e+17,6.320863e+16,9.139922e+16,7.002346e+16,1.091333e+16,1.568367e+16,1.059777e+17,5.855360e+16,1.864731e+16,6.426096e+13,5.260927e+16,3.828075e+16,54.517972,20
2,Normalize,200.425661,5.660191e+15,9.511216e+14,1.375315e+15,1.053667e+15,1.642166e+14,2.359975e+14,1.594681e+15,8.810757e+14,2.805924e+14,9.669564e+11,7.916294e+14,5.760233e+14,1.693023,20
3,RandomHorizontalFlip,81.057167,4.804517e+16,8.073367e+15,1.167403e+16,8.943796e+15,1.393913e+15,2.003208e+15,1.353607e+16,7.478800e+15,2.381741e+15,8.207777e+12,6.719556e+15,4.889436e+15,0.684701,20
4,RandomResizedCrop,1600.039134,9.601601e+16,1.613424e+16,2.333000e+16,1.787375e+16,2.785669e+15,4.003317e+15,2.705120e+16,1.494603e+16,4.759797e+15,1.640285e+13,1.342872e+16,9.771306e+15,13.515749,20


In [36]:
# print columns of combined_df
combined_df.columns

Index(['Function', 'CPU Time (s)', 'Retiring', 'Front-End Bound',
       'Bad Speculation', 'L1 Bound', 'L2 Bound', 'L3 Bound',
       'Memory Bandwidth', 'Local DRAM', 'Remote DRAM', 'Remote Cache',
       'Store Bound', 'Core Bound', 'CPU Time %', 'config'],
      dtype='object')

In [38]:
normalize_funcs = ['CPU Time (s)', 'Retiring', 'Front-End Bound',
       'Bad Speculation', 'L1 Bound', 'L2 Bound', 'L3 Bound',
       'Memory Bandwidth', 'Local DRAM', 'Remote DRAM', 'Remote Cache',
       'Store Bound', 'Core Bound']
# normalize values of these columns by dividing by the smallest sum of functions per config
for col in normalize_funcs:
       print('Normalizing: ',col)
       # get the sum of the config
       sum_col = combined_df.groupby('config')[col].sum().min()
       # print('Sum: ',sum_col)
       # if sum is 0, skip
       if sum_col == 0:
              continue
       # get lowest sum and divide by it
       # if only one config, then don't divide
       if type(sum_col) == float:
              combined_df[col] = combined_df[col]
       else:
              combined_df[col] = combined_df[col].div(sum_col.min())

Normalizing:  CPU Time (s)
Normalizing:  Retiring
Normalizing:  Front-End Bound
Normalizing:  Bad Speculation
Normalizing:  L1 Bound
Normalizing:  L2 Bound
Normalizing:  L3 Bound
Normalizing:  Memory Bandwidth
Normalizing:  Local DRAM
Normalizing:  Remote DRAM
Normalizing:  Remote Cache
Normalizing:  Store Bound
Normalizing:  Core Bound


In [39]:
combined_df.head(len(combined_df))

,Function,CPU Time (s),Retiring,Front-End Bound,Bad Speculation,L1 Bound,L2 Bound,L3 Bound,Memory Bandwidth,Local DRAM,Remote DRAM,Remote Cache,Store Bound,Core Bound,CPU Time %,config
0,Collation,206.538240,2.594403e+16,4.359557e+15,6.303888e+15,4.829582e+15,7.527024e+14,1.081717e+15,7.309376e+15,4.038495e+15,1.286122e+15,4.432138e+12,3.628510e+15,2.640258e+15,1.744657,20
1,Loader,6454.017994,3.761590e+17,6.320863e+16,9.139922e+16,7.002346e+16,1.091333e+16,1.568367e+16,1.059777e+17,5.855360e+16,1.864731e+16,6.426096e+13,5.260927e+16,3.828075e+16,54.517972,20
2,Normalize,200.425661,5.660191e+15,9.511216e+14,1.375315e+15,1.053667e+15,1.642166e+14,2.359975e+14,1.594681e+15,8.810757e+14,2.805924e+14,9.669564e+11,7.916294e+14,5.760233e+14,1.693023,20
3,RandomHorizontalFlip,81.057167,4.804517e+16,8.073367e+15,1.167403e+16,8.943796e+15,1.393913e+15,2.003208e+15,1.353607e+16,7.478800e+15,2.381741e+15,8.207777e+12,6.719556e+15,4.889436e+15,0.684701,20
4,RandomResizedCrop,1600.039134,9.601601e+16,1.613424e+16,2.333000e+16,1.787375e+16,2.785669e+15,4.003317e+15,2.705120e+16,1.494603e+16,4.759797e+15,1.640285e+13,1.342872e+16,9.771306e+15,13.515749,20
5,ToTensor,720.387803,3.856345e+17,6.480086e+16,9.370157e+16,7.178736e+16,1.118824e+16,1.607875e+16,1.086472e+17,6.002857e+16,1.911704e+16,6.587970e+13,5.393450e+16,3.924504e+16,6.085214,20


In [40]:
for col in combined_df.columns:
    if col == 'config' or col == 'Function':
        continue
    print(col)
    # pivot table
    df_pivot = combined_df.pivot(index='config', columns='Function', values=col)
    # sort the index using natsort
    df_pivot = df_pivot.reindex(natsort.natsorted(df_pivot.index))
    # find lowest value in entire dataframe and normalize the dataframe by dividing each value by the lowest value suhc that its not 0
    ax = df_pivot.plot(kind='bar', stacked=True, figsize=(100,60),colormap='jet')
    # for c in ax.containers:
    #     ax.bar_label(c, label_type='center')
    # make x axis label horizontal
    plt.xticks(rotation=0)
    # add x axis label
    plt.xlabel('Dataloaders')
    # add y axis label
    plt.ylabel(f"{col}")
    # add title to legend
    plt.title("Hardware metric breakdown by preprocessing operations")    
    # reverse legend order
    handles, labels = plt.gca().get_legend_handles_labels()
    plt.legend(handles[::-1], labels[::-1],loc='center left', bbox_to_anchor=(1.0, 0.5))
    plt.subplots_adjust(right=0.70)
    plt.savefig(os.path.join(plot_dir,col+'.png'))
    plt.close()


CPU Time (s)
Retiring
Front-End Bound
Bad Speculation
L1 Bound
L2 Bound
L3 Bound
Memory Bandwidth
Local DRAM
Remote DRAM
Remote Cache
Store Bound
Core Bound
CPU Time %
